##***I варіант***

Легковий автомобіль служби доставки протягом дня має відвідати усі магазини Епіцентр міста Києва. Розглядаємо довільний будній день, точка старту - Південний залізничний вокзал. Час перебування у кожному магазині - 10 хвилин.

Запропонуйте спосіб визначення оптимальної послідовності відвідувань магазинів та часу відправлення (як критерій оптимальності приймається загальний час у дорозі). За можливості продемонструйте план оптимальної поїздки (час прибуття та відправлення для кожного магазину).

***Ключове слово-підказка***: TomTom

***Очікуване рішення***: знайти Waypoint Optimization API, врахувати його обмеження у 12 точок - об'єднати кілька поруч розташованих магазинів в одну точку, запустити з різним часом відправлення, обрати найкращий варіант.

In [3]:
import requests
import time
from datetime import datetime, timedelta
import folium

# API Constants Definition
base_url = "https://api.tomtom.com"
endpoint = "/routing/waypointoptimization/1"
api_key = "dZj2xYGtfCc9gsIYqUQ3xk7FADJ3fJkk"

# Example route points data
waypoints = [
    {"point": {"latitude": 50.4406, "longitude": 30.4989}},  # Pivdennyi railway station, Kyiv
    {"point": {"latitude": 50.49330067738287, "longitude": 30.362166966807383}},  # Berkovetska Street, 6B
    {"point": {"latitude": 50.52293904046261, "longitude": 30.479614266871618}},  # Poliarna Street, 20D
    {"point": {"latitude": 50.49415943365886, "longitude": 30.48555067426719}},  # Stepana Bandery Avenue, 11A
    {"point": {"latitude": 50.48804364083631, "longitude": 30.527092708697932}},  # Stepana Bandery Avenue, 36A
    {"point": {"latitude": 50.44280589532593, "longitude": 30.353028148191637}},  # Kryshaleva Street, 6
    {"point": {"latitude": 50.44806835017768, "longitude": 30.44418519213106}},  # Borschahivska Street, 154
    {"point": {"latitude": 50.45505769279923, "longitude": 30.47454168477855}},  # Berestey Street, 22
    {"point": {"latitude": 50.38133144941649, "longitude": 30.447164923038784}},  # Kiltseva Road, 1B
    {"point": {"latitude": 50.492250453701914, "longitude": 30.611229406053766}},  # Bratislavska Street, 11
    {"point": {"latitude": 50.4567773575752, "longitude": 30.651194680496364}},  # Viskozna Street, 5
    {"point": {"latitude": 50.39217885943812, "longitude": 30.632464644528582}},  # Petra Hryhorenka Avenue, 40
    {"point": {"latitude": 50.42534844873109, "longitude": 30.595504591736805}},  # Dniprovskaya Embankment, 13-V
    {"point": {"latitude": 50.438451709164156, "longitude": 30.619906583799594}}  # Sobornosti Avenue, 7B
]

# Example request options
options = {
    "travelMode": "car",
    "traffic": "live"
}

# Split route points into chunks based on the maximum allowed (12 points)
chunk_size = 12
waypoint_chunks = [waypoints[i:i + chunk_size] for i in range(0, len(waypoints), chunk_size)]

# List of real addresses corresponding to each route point
addresses = [
    "Pivdennyi railway station",
    "Berkovetska Street, 6B",
    "Poliarna Street, 20D",
    "Stepana Bandery Avenue, 11A",
    "Stepana Bandery Avenue, 36A",
    "Kryshaleva Street, 6",
    "Borschahivska Street, 154",
    "Berestey Street, 22",
    "Kiltseva Road, 1B",
    "Bratislavska Street, 11",
    "Viskozna Street, 5",
    "Petra Hryhorenka Avenue, 40",
    "Dniprovskaya Embankment, 13-V",
    "Sobornosti Avenue, 7B"
]

# Departure times for each chunk
departure_times = [
    "2024-06-18T07:00:00Z",
    "2024-06-18T07:30:00Z",
    "2024-06-18T08:00:00Z",
    "2024-06-18T08:30:00Z"
]

# Delay between requests (in seconds) to avoid 429 errors
delay_between_requests = 1.5


def calculate_travel_time(coord1, coord2):
    """Function to calculate travel time between two points"""
    # Emulate time calculation based on distance (here simply use 5-15 minutes)
    distance_km = calculate_distance(coord1, coord2)
    if distance_km < 5:
        return 5
    elif distance_km < 10:
        return 10
    else:
        return 15


def calculate_distance(coord1, coord2):
    """Function to emulate distance calculation between two points"""
    return 10  # Emulate distance in kilometers


# Create a folium map object
m = folium.Map(location=[50.45, 30.52], zoom_start=12)  # Initial coordinates and map zoom level

# Add route points to the map
for point in waypoints:
    folium.Marker(
        location=[point['point']['latitude'], point['point']['longitude']],
        popup='Address',
        icon=folium.Icon(icon='cloud')
    ).add_to(m)

# Request to the TomTom API to get the optimal route
optimized_route_coords = []

for i, chunk in enumerate(waypoint_chunks):
    request_body = {
        "waypoints": chunk,
        "options": options
    }

    if i < len(departure_times):
        request_body["options"]["departAt"] = departure_times[i]

    try:
        # Send POST request with JSON body
        response = requests.post(f"{base_url}{endpoint}?key={api_key}", json=request_body)

        # Check if request is successful (status code 200)
        if response.status_code == 200:
            # Process and output response data
            data = response.json()
            optimized_order = data.get('optimizedOrder')
            if optimized_order:
                print(f"Optimal order of points for chunk {i + 1}: {optimized_order}")
                print("Addresses:")
                total_time = datetime.strptime(departure_times[i], "%Y-%m-%dT%H:%M:%SZ")
                for idx, index in enumerate(optimized_order):
                    # Check if index is within the addresses list bounds
                    if 0 <= index < len(addresses):
                        travel_time = calculate_travel_time(chunk[idx]["point"], chunk[idx + 1]["point"]) if idx < len(chunk) - 1 else 0
                        arrival_time = total_time + timedelta(minutes=travel_time)
                        departure_time = arrival_time + timedelta(minutes=10)  # Time for visiting
                        print(f"Point {index}: {addresses[index]}")
                        print(f"Arrival: {arrival_time.strftime('%Y-%m-%d %H:%M:%S')}")
                        print(f"Departure: {departure_time.strftime('%Y-%m-%d %H:%M:%S')}")
                        print()
                        total_time = departure_time  # Update total departure time
                        # Add the coordinates of the optimal route
                        point = chunk[index]["point"]
                        coord = [point["latitude"], point["longitude"]]
                        optimized_route_coords.append(coord)
                print()
            else:
                print(f"No optimal route found for chunk {i + 1}.")

        else:
            # Print error message for status code other than 200
            print(f"Error: {response.status_code} - {response.text}")

        # Pause between requests to avoid API limits
        time.sleep(delay_between_requests)

    except requests.exceptions.RequestException as e:
        # Handle network errors
        print("Network error:", e)

folium.PolyLine(
    locations=optimized_route_coords,
    color='blue',
    weight=5,
    tooltip='Optimized Route'
).add_to(m)

m.save('optimized_route_map.html')

Optimal order of points for chunk 1: [0, 7, 6, 8, 5, 1, 2, 3, 4, 9, 10, 11]
Addresses:
Point 0: Pivdennyi railway station
Arrival: 2024-06-18 07:15:00
Departure: 2024-06-18 07:25:00

Point 7: Berestey Street, 22
Arrival: 2024-06-18 07:40:00
Departure: 2024-06-18 07:50:00

Point 6: Borschahivska Street, 154
Arrival: 2024-06-18 08:05:00
Departure: 2024-06-18 08:15:00

Point 8: Kiltseva Road, 1B
Arrival: 2024-06-18 08:30:00
Departure: 2024-06-18 08:40:00

Point 5: Kryshaleva Street, 6
Arrival: 2024-06-18 08:55:00
Departure: 2024-06-18 09:05:00

Point 1: Berkovetska Street, 6B
Arrival: 2024-06-18 09:20:00
Departure: 2024-06-18 09:30:00

Point 2: Poliarna Street, 20D
Arrival: 2024-06-18 09:45:00
Departure: 2024-06-18 09:55:00

Point 3: Stepana Bandery Avenue, 11A
Arrival: 2024-06-18 10:10:00
Departure: 2024-06-18 10:20:00

Point 4: Stepana Bandery Avenue, 36A
Arrival: 2024-06-18 10:35:00
Departure: 2024-06-18 10:45:00

Point 9: Bratislavska Street, 11
Arrival: 2024-06-18 11:00:00
Departure:

In [4]:
m